In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:



df = pd.read_csv('injury-details-2025-q3.csv')


london_boroughs = [
    'Barking & Dagenham', 'Barnet', 'Bexley', 'Brent', 'Bromley',
    'Camden', 'Croydon', 'Ealing', 'Enfield', 'Greenwich',
    'Hackney', 'Hammersmith & Fulham', 'Haringey', 'Harrow',
    'Havering', 'Hillingdon', 'Hounslow', 'Islington',
    'Kensington & Chelsea', 'Kingston Upon Thames', 'Lambeth',
    'Lewisham', 'Merton', 'Newham', 'Redbridge',
    'Richmond Upon Thames', 'Southwark', 'Sutton',
    'Tower Hamlets', 'Waltham Forest', 'Wandsworth', 'Westminster',
    'City Of London'
]


df_london = df[df['Borough'].isin(london_boroughs)].copy()



borough_counts = df_london['Borough'].value_counts().reset_index()
borough_counts.columns = ['Borough', 'number_of_incidents']


severe_mask = df_london['Injury outcome'].astype(str).str.contains('taken to hospital', case=False, na=False)
severe_counts = df_london[severe_mask]['Borough'].value_counts().reset_index()
severe_counts.columns = ['Borough', 'number_of_severe_incidents']


result = pd.merge(borough_counts, severe_counts, on='Borough', how='left')
result['number_of_severe_incidents'] = result['number_of_severe_incidents'].fillna(0).astype(int)
result = result.sort_values('number_of_incidents', ascending=False).reset_index(drop=True)
result.to_csv('CSVForQuestion1.csv', index=False)





In [1]:
import geopandas as gpd
import pandas as pd

gdf = gpd.read_file("LondonGeo.json", driver='TopoJSON', layer='boroughs')

gdf['longitude'] = gdf.centroid.x
gdf['latitude'] = gdf.centroid.y

centroids = gdf[['id', 'longitude', 'latitude']].copy()
centroids.columns = ['Borough', 'longitude', 'latitude']

incidents = pd.read_csv("CSVForQuestion1.csv")
merged = incidents.merge(centroids, on='Borough', how='left')

# Make absolutely sure no index is saved
merged.to_csv("Centroids.csv", index=False)  # ✓ You had this

# But verify the output
with open('Centroids.csv', 'r') as f:
    print("First line:", f.readline())


First line: Borough,number_of_incidents,number_of_severe_incidents,longitude,latitude



c:\Users\mason\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\pyogrio\raw.py:200: RuntimeWarning: driver TopoJSON does not support open option DRIVER
  return ogr_read(


In [2]:

df = pd.read_csv('injury-details-2025-q3.csv')
df.head(10)

,Incident type,Date,Route,Operator,Borough,Garage,Injury outcome,Victim category,Age,Gender,Injury,ID
0,Assault,2014-01-01,143,Metroline,Haringey,Cricklewood,Other,Bus Driver,Unknown,Unknown,Yes,39953.0
1,Collision,2014-01-01,217,Metroline,Broxbourne,Potters Bar,Taken to hospital,Pedestrian,Adult (18-65),Male,Yes,39741.0
2,Collision,2014-01-01,N86,Stagecoach,Havering,Romford (East London Buses),Other,Pedestrian,Adult (18-65),Male,Yes,39773.0
3,Slip Trip Fall,2014-01-01,430,Go-Ahead,Wandsworth,Putney,Taken to hospital,Passenger / customer,Adult (18-65),Male,Yes,39727.0
4,Slip Trip Fall,2014-01-01,N35,Transport UK,Southwark,Walworth,Other,Passenger / customer,Adult (18-65),Female,Yes,39730.0
5,Slip Trip Fall,2014-01-01,7,Metroline,Westminster,West Perivale,Refused treatment,Passenger / customer,Adult (18-65),Female,Yes,39735.0
6,Slip Trip Fall,2014-01-01,9,First Bus,Kensington & Chelsea,Stamford Brook,Refused treatment,Passenger / customer,Child (0-12),Unknown,Yes,39740.0
7,Slip Trip Fall,2014-01-01,414,Transport UK,Westminster,Battersea (Abellio London),Other,Passenger / customer,Adult (18-65),Female,Yes,39774.0
8,Slip Trip Fall,2014-01-01,27,First Bus,Camden,Stamford Brook,Refused treatment,Passenger / customer,Adult (18-65),Male,Yes,39777.0
9,Collision,2014-01-02,384,Metroline,Barnet,Potters Bar,Taken to hospital,Pedestrian,Child (0-12),Male,Yes,39751.0
